In [ ]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [ ]:
# Import 
File_1_URL = "data/PLACEHOLDER.csv"
File_1_df = pd.read_csv(File_1_URL)
File_1_df.count()

In [ ]:
File_2_URL = "data/PLACEHOLDER.csv"
File_2_df = pd.read_csv(File_2_URL)
File_2_df.count()

In [ ]:
File_3_URL = "data/PLACEHOLDER.csv"
File_3_df = pd.read_csv(File_3_URL)
File_3_df.count()

## Transform Data

In [ ]:
# Transform DataSet 1

In [ ]:
# Transform DataSet 2


In [ ]:
# Transform DataSet 3


### Create database connection

In [ ]:
db_url = "postgres:postgres@localhost:5432/covid_census_db"
engine = create_engine(f'postgresql://{db_url}')
engine.table_names()

### Load DataFrames into database

In [ ]:
File_1_df.to_sql(name='PLACEHOLDER1', con=engine, if_exists='append', index=True)

In [ ]:
File_2_df.to_sql(name='PLACEHOLDER2', con=engine, if_exists='append', index=True)

In [ ]:
File_3_df.to_sql(name='PLACEHOLDER3', con=engine, if_exists='append', index=True)
